# Practical 4 &ndash; Mixed-Integer Linear Programming in JuMP: The facility location problem


---

Inspired from [a JuMP tutorial](https://jump.dev/JuMP.jl/v1.10.0/tutorials/linear/facility_location/)

In [ ]:
using JuMP
import HiGHS
import LinearAlgebra
import Plots
import Random

## Uncapacitated facility location

### Problem description

We are given
* A set $M=\{1, \dots, m\}$ of clients
* A set $N=\{ 1, \dots, n\}$ of sites where a facility can be built

**Decision variables**
Decision variables are split into two categories:
* Binary variable $y_{j}$ indicates whether facility $j$ is built or not
* Binary variable $x_{i, j}$ indicates whether client $i$ is assigned to facility $j$

**Objective**
The objective is to minimize the total cost of serving all clients.
This costs breaks down into two components:
* Fixed cost of building a facility.
In this example, this cost is $f_{j} = 1, \ \forall j$.
* Cost of serving clients from the assigned facility.
In this example, the cost $c_{i, j}$
of serving client $i$ from facility $j$
is the Euclidean distance between the two.

**Constraints**
* Each customer must be served by exactly one facility
* A facility cannot serve any client unless it is open

### MILP formulation

The problem can be formulated as the following MILP:

$$
\begin{aligned}
\min_{x, y} \ \ \ &
\sum_{i, j} c_{i, j} x_{i, j} +
\sum_{j} f_{j} y_{j} \\
s.t. &
\sum_{j} x_{i, j} = 1, && \forall i \in M \\
& x_{i, j} \leq y_{j}, && \forall i \in M, j \in N \\
& x_{i, j}, y_{j} \in \{0, 1\}, && \forall i \in M, j \in N
\end{aligned}
$$

where the first set of constraints ensures
that each client is served exactly once,
and the second set of constraints ensures
that no client is served from an unopened facility.

### Problem data

In [ ]:
Random.seed!(314)

number of clients

In [ ]:
m = 12

number of facility locations

In [ ]:
n = 5

Clients' locations

In [ ]:
Xc = rand(m)
Yc = rand(m)

Facilities' potential locations

In [ ]:
Xf = rand(n)
Yf = rand(n)

Fixed costs

In [ ]:
f = ones(n);

Distance

In [ ]:
c = zeros(m, n)
for i in 1:m
    for j in 1:n
        c[i, j] = LinearAlgebra.norm([Xc[i] - Xf[j], Yc[i] - Yf[j]], 2)
    end
end

Display the data

In [ ]:
Plots.scatter(
    Xc,
    Yc,
    label = "Clients",
    markershape = :circle,
    markercolor = :blue,
)
Plots.scatter!(
    Xf,
    Yf,
    label = "Facility",
    markershape = :square,
    markercolor = :white,
    markersize = 6,
    markerstrokecolor = :red,
    markerstrokewidth = 2,
)

### JuMP implementation


Create a JuMP model

In [ ]:
ufl = Model(HiGHS.Optimizer)

Variables

In [ ]:
@variable(ufl, y[1:n], Bin);
@variable(ufl, x[1:m, 1:n], Bin);

Each client is served exactly once

In [ ]:
@constraint(ufl, client_service[i in 1:m], sum(x[i, j] for j in 1:n) == 1);

A facility must be open to serve a client

In [ ]:
@constraint(ufl, open_facility[i in 1:m, j in 1:n], x[i, j] <= y[j]);

Objective

In [ ]:
@objective(ufl, Min, f'y + sum(c .* x));

Solve the uncapacitated facility location problem with HiGHS

In [ ]:
optimize!(ufl)
solution_summary(ufl)

### Visualizing the solution

The threshold 1e-5 ensure that edges between clients and facilities are drawn when x[i, j] ≈ 1.

In [ ]:
x_ = value.(x) .> 1 - 1e-5
y_ = value.(y) .> 1 - 1e-5

Display clients

In [ ]:
p = Plots.scatter(
    Xc,
    Yc,
    markershape = :circle,
    markercolor = :blue,
    label = nothing,
)

Show open facility

In [ ]:
mc = [(y_[j] ? :red : :white) for j in 1:n]
Plots.scatter!(
    Xf,
    Yf,
    markershape = :square,
    markercolor = mc,
    markersize = 6,
    markerstrokecolor = :red,
    markerstrokewidth = 2,
    label = nothing,
)

Show client-facility assignment

In [ ]:
for i in 1:m
    for j in 1:n
        if x_[i, j] == 1
            Plots.plot!(
                [Xc[i], Xf[j]],
                [Yc[i], Yf[j]],
                color = :black,
                label = nothing,
            )
        end
    end
end

p

## Homework: Capacitated facility location

### Problem formulation

The capacitated variant introduces a capacity constraint on each facility, i.e., clients have a certain level of demand to be served, while each facility only has finite capacity which cannot be exceeded.

Specifically,
* The demand of client $i$ is denoted by $a_{i} \geq 0$
* The capacity of facility $j$ is denoted by $q_{j} \geq 0$

The capacity constraints then write
$$
\begin{aligned}
\sum_{i} a_{i} x_{i, j} &\leq q_{j} y_{j} && \forall j \in N
\end{aligned}
$$

Note that, if $y_{j}$ is set to $0$, the capacity constraint above automatically forces $x_{i, j}$ to $0$.

Thus, the capacitated facility location can be formulated as follows

$$
\begin{aligned}
\min_{x, y} \ \ \ &
\sum_{i, j} c_{i, j} x_{i, j} +
\sum_{j} f_{j} y_{j} \\
s.t. &
\sum_{j} x_{i, j} = 1, && \forall i \in M \\
& \sum_{i} a_{i} x_{i, j} \leq q_{j} y_{j}, && \forall j \in N \\
& x_{i, j}, y_{j} \in \{0, 1\}, && \forall i \in M, j \in N
\end{aligned}
$$

For simplicity, we will assume that there is enough capacity to serve the demand,
 i.e., there exists at least one feasible solution.

In [ ]:
Random.seed!(314)

Demands

In [ ]:
a = rand(1:3, m);

Capacities

In [ ]:
q = rand(5:10, n);

Display the data

In [ ]:
Plots.scatter(
    Xc,
    Yc,
    label = nothing,
    markershape = :circle,
    markercolor = :blue,
    markersize = 2 .* (2 .+ a),
)

Plots.scatter!(
    Xf,
    Yf,
    label = nothing,
    markershape = :rect,
    markercolor = :white,
    markersize = q,
    markerstrokecolor = :red,
    markerstrokewidth = 2,
)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*